In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil import parser

import mysql.connector

# Functions
from uploadDB import insert_channel_in_DB, remove_emojis, remove_links_and_line_jumps, remove_accents, clean_video_statics, video_tables_db_upload

# Getting the files in the folder:

In [2]:
sheets = ['channel_evolution', 'video_statics', 'video_variables']

In [3]:
# The list of items
files = os.listdir("./extractions")
 
# Loop to print each filename separately
for filename in files:
    print(filename)

Extraction_2023-04-17.xlsx
Extraction_2023-04-18.xlsx
Extraction_2023-04-19.xlsx
Extraction_2023-04-20.xlsx
Extraction_2023-04-21.xlsx
Extraction_2023-04-24.xlsx
Extraction_2023-04-26.xlsx
Extraction_2023-04-27.xlsx
Extraction_2023-04-28.xlsx
Extraction_2023-05-01.xlsx


In [4]:
# Reading each file in dict
dump = {}

for i,filename in enumerate(files):

    print(filename)

    for sheet in sheets:

        print('--->', sheet)

        # First Read
        if i == 0:
            dump[sheet] = pd.read_excel('extractions/' + filename, sheet_name=sheet, index_col=[0])
        else:
            helpDF = pd.read_excel('extractions/' + filename, sheet_name=sheet,index_col=[0])

            dump[sheet] = pd.concat([dump[sheet], helpDF], ignore_index=False)

Extraction_2023-04-17.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-04-18.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-04-19.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-04-20.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-04-21.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-04-24.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-04-26.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-04-27.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-04-28.xlsx
---> channel_evolution
---> video_statics
---> video_variables
Extraction_2023-05-01.xlsx
---> channel_evolution
---> video_statics
---> video_variables


In [5]:
insert_channel_in_DB(dump, chop_details = True)

Inserted: channel_evolution


Video Level

In [6]:
dump2 = {
    'video_statics' : dump['video_statics'].reset_index().copy(),
    'video_variables' : dump['video_variables'].reset_index().copy()
}

# Keeping only one version of the extractions
dump2['video_statics'].drop_duplicates(subset=['ChannelID','video_id','CreatedOn'], keep='first', inplace=True)
dump2['video_statics'].drop_duplicates(subset=['ChannelID','video_id'], keep='first', inplace=True)

In [7]:
# Get all the video_statics already in the DB to clean

# Connect to the database
cnx = mysql.connector.connect(host='localhost',
                                        database='youtubeproject',
                                        user='root',
                                        password='optim123*')

sql_query = """SELECT video_id from video_statics"""
frstatics = pd.read_sql(sql_query, cnx)
vids_in_statics = frstatics['video_id'].to_list()

cnx.close()

# Filtering
cleaning_mask = ~dump2['video_statics']['video_id'].isin( vids_in_statics )

dump2['video_statics'] = dump2['video_statics'][cleaning_mask]
#dump2['video_statics']

C:\Users\maike\AppData\Local\Temp\ipykernel_17112\3968575264.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  frstatics = pd.read_sql(sql_query, cnx)


,ChannelID,video_id,CreatedOn,title,descr,duration
0,UCo8ifS6JKfDqV33pwAMeV0Q,biFmKCvch0o,2023-04-17,🔥ESTRENO🔥 15 REGLAS para HOMBRES 😎 Nacho Redon...,00:00:00 Introducción\n00:01:27 Nacho Redondo ...,4506
1,UCo8ifS6JKfDqV33pwAMeV0Q,fV0W-G5SlY4,2023-04-16,Cuando Adidas te manda regalos.,NaN,60
600,UCYbtAAKfp11tWhTjEcrvyAQ,cXVVZyYdlSg,2023-04-16,La revancha con Irrael ¿quién fue el campeón d...,Esto se descontroló. Aún no nos recuperamos de...,4839
601,UCYbtAAKfp11tWhTjEcrvyAQ,mzbXuLH_kVw,2023-04-15,Entregados a EntreGrados EP 02 | Versión madre...,¿Te imaginas beber con tu mamá? O al revés ¿be...,1262
1074,UC7Iabt-Qo5I5c8-cnwTHiWg,8HDIcLQcbxE,2023-04-14,Labias De Parrilla: Los Relojes | 021,En este episodio hablamos del origen y el func...,855
...,...,...,...,...,...,...
38522,UCZbq1qvbQzxDGUyuoPnF4LQ,cVY8c1esbIc,2023-04-29,¿Cuánto costaría nuestro s3m3n?,NaN,42
38523,UCZbq1qvbQzxDGUyuoPnF4LQ,QQ9M8JlfTIc,2023-04-29,¿Qué es un NPC?,NaN,48
38524,UCZbq1qvbQzxDGUyuoPnF4LQ,0mgaxFplwhA,2023-04-28,Quién es ADOPTADO en #escueladenada,NaN,43
38525,UCZbq1qvbQzxDGUyuoPnF4LQ,azE8FGFPUEE,2023-04-28,Ser CULERO por no compartir mi comida,NaN,52


In [8]:
# cleaning here instead on function
dump2['video_statics']['title'] = dump2['video_statics']['title'].apply(remove_emojis).apply(remove_links_and_line_jumps).apply(remove_accents)
dump2['video_statics']['descr'] = dump2['video_statics']['descr'].astype(str).apply(remove_emojis).apply(remove_links_and_line_jumps).apply(remove_accents)

#dump2['video_statics']

In [9]:
video_tables_db_upload( dump2 )

d:\Matematica\Data Science\_PersonalProjects\EDN Dashboard\uploadDB.py:117: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  video_ids_in_db = pd.read_sql("SELECT video_id FROM video_statics", cnx)['video_id'].to_list()


Inserting: video_statics...
Inserted: video_statics
Inserting: video_variables...
Inserted: video_variables
